---
title: "Multinomial Logit Model"
author: "Timon Ortwein"
date: today
---


This assignment expores two methods for estimating the MNL model: (1) via Maximum Likelihood, and (2) via a Bayesian approach using a Metropolis-Hastings MCMC algorithm. 


## 1. Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 3 products, then either $y=3$ or $y=(0,0,1)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, brand, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 3 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^3e^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$



## 2. Simulate Conjoint Data

We will simulate data from a conjoint experiment about video content streaming services. We elect to simulate 100 respondents, each completing 10 choice tasks, where they choose from three alternatives per task. For simplicity, there is not a "no choice" option; each simulated respondent must select one of the 3 alternatives. 

Each alternative is a hypothetical streaming offer consistent of three attributes: (1) brand is either Netflix, Amazon Prime, or Hulu; (2) ads can either be part of the experience, or it can be ad-free, and (3) price per month ranges from \$4 to \$32 in increments of \$4.

The part-worths (ie, preference weights or beta parameters) for the attribute levels will be 1.0 for Netflix, 0.5 for Amazon Prime (with 0 for Hulu as the reference brand); -0.8 for included adverstisements (0 for ad-free); and -0.1*price so that utility to consumer $i$ for hypothethical streaming service $j$ is 

$$
u_{ij} = (1 \times Netflix_j) + (0.5 \times Prime_j) + (-0.8*Ads_j) - 0.1\times Price_j + \varepsilon_{ij}
$$

where the variables are binary indicators and $\varepsilon$ is Type 1 Extreme Value (ie, Gumble) distributed.

The following code provides the simulation of the conjoint data.

:::: {.callout-note collapse="true"}

In [ ]:
import numpy as np
import pandas as pd

# set seed for reproducibility
np.random.seed(123)

# define attributes
brand = ["N", "P", "H"]  # Netflix, Prime, Hulu
ad = ["Yes", "No"]
price = list(range(8, 33, 4))

# generate all possible profiles
profiles = pd.DataFrame([
    (b, a, p) for b in brand for a in ad for p in price
], columns=["brand", "ad", "price"])
m = len(profiles)

# assign part-worth utilities (true parameters)
b_util = {"N": 1.0, "P": 0.5, "H": 0}
a_util = {"Yes": -0.8, "No": 0.0}
def p_util(p): return -0.1 * p

# number of respondents, choice tasks, and alternatives per task
n_peeps = 100
n_tasks = 10
n_alts = 3

# function to simulate one respondent's data
def sim_one(id):
    datlist = []
    for t in range(1, n_tasks + 1):
        dat = profiles.sample(n=n_alts).copy()
        dat.insert(0, "task", t)
        dat.insert(0, "resp", id)
        dat["v"] = (
            dat["brand"].map(b_util) +
            dat["ad"].map(a_util) +
            dat["price"].apply(p_util)
        ).round(10)
        dat["e"] = -np.log(-np.log(np.random.uniform(size=n_alts)))
        dat["u"] = dat["v"] + dat["e"]
        dat["choice"] = (dat["u"] == dat["u"].max()).astype(int)
        datlist.append(dat)
    return pd.concat(datlist, ignore_index=True)

# simulate data for all respondents
conjoint_data = pd.concat([sim_one(i) for i in range(1, n_peeps + 1)], ignore_index=True)

# remove values unobservable to the researcher
conjoint_data = conjoint_data[["resp", "task", "brand", "ad", "price", "choice"]]

# clean up
for name in dir():
    if name != "conjoint_data":
        del globals()[name]

print(conjoint_data.head())

::::



## 3. Preparing the Data for Estimation

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). The fact that each task for each respondent has the same number of alternatives (3) helps.  In addition, we need to convert the categorical variables for brand and ads into binary variables.

In [ ]:
conjoint_data['brand_netflix'] = (conjoint_data['brand'] == 'N').astype(int)
conjoint_data['brand_prime'] = (conjoint_data['brand'] == 'P').astype(int)
conjoint_data['ads_yes'] = (conjoint_data['ad'] == 'Yes').astype(int)

X = conjoint_data[['brand_netflix', 'brand_prime', 'ads_yes', 'price']].values
y = conjoint_data['choice'].values

respondent = conjoint_data['resp'].values
task = conjoint_data['task'].values

print(conjoint_data.head())

## 4. Estimation via Maximum Likelihood

Following you can see the code for the MNL.

In [ ]:
import numpy as np

def mnl(beta, X, y, ids):
    v = X @ beta
    
    unique_tasks = np.unique(ids)
    log_likelihood = 0.0
    for t in unique_tasks:
        idx = (ids == t)
        v_t = v[idx]
        y_t = y[idx]

        denom = np.log(np.sum(np.exp(v_t)))
        log_likelihood += np.sum(y_t * (v_t - denom))

    return -log_likelihood

task_ids = conjoint_data['resp'].astype(str) + "_" + conjoint_data['task'].astype(str)
task_ids = task_ids.values

In the next step we minimize using optimize to get the MLE estimates, standard error and 95% CIs.

In [ ]:
from scipy.optimize import minimize
import pandas as pd

result = minimize(
    mnl,
    x0=np.zeros(4),
    args=(X, y, task_ids),
    method='BFGS'
)

hessian = result.hess_inv
error = np.sqrt(np.diag(hessian))
ci_lower = result.x - 1.96 * error
ci_upper = result.x + 1.96 * error

results_df = pd.DataFrame({
    'Parameter': ['Netflix', 'Prime', 'Ads', 'Price'],
    'MLE_Estimate': result.x,
    'Standard_Error': error,
    'CI_Lower': ci_lower,
    'CI_Upper': ci_upper
})

print(results_df)

## 5. Estimation via Bayesian Methods

We implement a Metropolis-Hastings MCMC sampler to estimate the posterior distribution of the parameters. The sampler uses:
- 11,000 total steps with 1,000 burn-in iterations
- N(0,5) priors for binary variable coefficients
- N(0,1) prior for the price coefficient
- Multivariate normal proposal distribution


In [ ]:
import numpy as np

def log_prior(beta):
    lp = 0
    lp += -0.5 * ((beta[0] / 5) ** 2 + np.log(2 * np.pi * 25))
    lp += -0.5 * ((beta[1] / 5) ** 2 + np.log(2 * np.pi * 25))
    lp += -0.5 * ((beta[2] / 5) ** 2 + np.log(2 * np.pi * 25))
    lp += -0.5 * ((beta[3] / 1) ** 2 + np.log(2 * np.pi * 1))
    return lp

def log_posterior(beta, X, y, task_ids):
    ll = -mnl(beta, X, y, task_ids)
    lp = log_prior(beta)
    return ll + lp

n_steps = 11000
burn_in = 1000
np.random.seed(42)

beta_current = np.zeros(4)
samples = np.zeros((n_steps, 4))
log_post_current = log_posterior(beta_current, X, y, task_ids)

proposal_sd = np.array([0.05, 0.05, 0.05, 0.005])

for step in range(n_steps):
    beta_proposal = beta_current + np.random.normal(0, proposal_sd)
    log_post_proposal = log_posterior(beta_proposal, X, y, task_ids)

    accept_prob = np.exp(log_post_proposal - log_post_current)
    if np.random.rand() < accept_prob:
        beta_current = beta_proposal
        log_post_current = log_post_proposal
    samples[step, :] = beta_current

posterior_samples = samples[burn_in:, :]

means = posterior_samples.mean(axis=0)
stds = posterior_samples.std(axis=0)
ci_lower = np.percentile(posterior_samples, 2.5, axis=0)
ci_upper = np.percentile(posterior_samples, 97.5, axis=0)

For the Netflix parameter, I visualize both the trace plot and histogram of the posterior distribution. The trace plot helps assess convergence of the MCMC chain by showing how the parameter value evolves over iterations. The histogram shows the shape and spread of the posterior distribution, giving us insight into the uncertainty around our parameter estimate.


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(posterior_samples[:,0])
plt.title("Trace plot: beta_Netflix")
plt.xlabel("Iteration")
plt.ylabel("Value")

plt.subplot(1,2,2)
plt.hist(posterior_samples[:,0], bins=30, density=True)
plt.title("Posterior: beta_Netflix")
plt.xlabel("Value")
plt.ylabel("Density")
plt.tight_layout()
plt.show()

Looking at the results from both estimation approaches, we can compare the posterior means, standard deviations, and intervals from our Bayesian analysis with the maximum likelihood estimates. 

In [ ]:
summary_df = pd.DataFrame({
    'Parameter': ['Netflix', 'Prime', 'Ads', 'Price'],
    'MLE_Estimate': result.x,
    'MLE_StdErr': error,
    'MLE_95CI_Lower': ci_lower,
    'MLE_95CI_Upper': ci_upper,
    'Bayes_Mean': means,
    'Bayes_StdDev': stds,
    'Bayes_95CI_Lower': ci_lower,
    'Bayes_95CI_Upper': ci_upper
})

print(summary_df)

## 6. Discussion

Looking at the parameter estimates, we can draw several meaningful insights about consumer preferences for streaming services:

Brand Preferences:
Consumers generally prefer Netflix and Amazon Prime over Hulu, which serves as the reference brand in this analysis. The positive coefficients for both Netflix and Prime indicate a higher utility associated with these services compared to Hulu. Furthermore, the coefficient for Netflix is larger than that of Prime, suggesting that, all else equal, consumers have a stronger preference for Netflix. This result is consistent with real-world market dynamics, where Netflix has historically maintained a dominant position in the streaming industry.

Price:
The negative coefficient for price aligns with economic intuition as the monthly subscription price increases, consumer utility and the probability of choosing that option decrease. The magnitude of this coefficient quantifies how sensitive consumers are to price changes. For each one-dollar increase in price, utility decreases by β_price units. Understanding this tradeoff enables streaming services to optimize their pricing strategies by balancing potential revenue gains from higher prices against potential subscriber losses.

Advertisement Impact:
The coefficient for ads is also negative, indicating that consumers generally prefer ad-free experiences. The magnitude of this coefficient suggests that the presence of ads significantly reduces consumer utility, an effect comparable to that of a substantial price increase. This finding underscores the importance of ad-free options in maintaining consumer satisfaction and subscription rates.


Simulation Changes 

Instead of assuming that all consumers are the same, the simulation can be made more realistic by allowing each person to have their own unique preferences. Instead of using one set of preferences (like how much people dislike ads or how sensitive they are to price), the model gives each individual their own version of these preferences.

These personal preferences would be based on general trends seen across all consumers, but with room for variation. This means some people might be more sensitive to price, others might care more about having no ads, and some might have a stronger preference for a particular streaming service.

By doing this, the simulation can better reflect real-life behavior, where different people value things differently.
